In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "/content/drive/My Drive/PseudoDataTrain"

CATEGORIES = ["type_1", "type_2", "type_3"]

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


"\nfor category in CATEGORIES:  # do dogs and cats\n    path = os.path.join(DATADIR,category)  # create path to dogs and cats\n    for img in os.listdir(path):  # iterate over each image per dogs and cats\n        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array\n        plt.imshow(img_array, cmap='gray')  # graph it\n        plt.show()  # display!\n\n        IMG_SIZE = 450\n\n        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))\n        plt.imshow(new_array, cmap='gray')\n        plt.show()\n\n        break  # we just want one for now so break\n    break  #...and one more!\n"

In [2]:
training_data = []
IMG_SIZE = 450

def create_training_data():
  for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)  
    class_num = CATEGORIES.index(category)  # if 0, type 1 data; 1, type 2 data; 2, type 3 data. 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])

create_training_data()

print(len(training_data))

600


In [3]:
DATADIR_VAL = "/content/drive/My Drive/PseudoDataValid"

validation_data = []

def create_validation_data():
  for category in CATEGORIES:  
    path = os.path.join(DATADIR_VAL,category)  
    class_num = CATEGORIES.index(category)  # if 0, type 1 data; 1, type 2 data; 3, type 3 data. 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        validation_data.append([new_array, class_num])

create_validation_data()

print(len(validation_data))

375


In [0]:
import random 

random.shuffle(training_data)
random.shuffle(validation_data)


In [0]:
X = []
y = []

X_val = []
y_val = []


In [0]:
for features, label in training_data:
  X.append(features)
  y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

for features, label in validation_data:
  X_val.append(features)
  y_val.append(label)

X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)




In [0]:
import pickle

pickle_out = open('X.pickle', 'wb')
pickle.dump(X, pickle_out)  # dumps X into pickle_out
pickle_out.close()

pickle_out = open('y.pickle', 'wb')
pickle.dump(y, pickle_out)  # dumps y into pickle_out
pickle_out.close()

pickle_out = open('X_val.pickle', 'wb')
pickle.dump(X_val, pickle_out)  # dumps X_val into pickle_out
pickle_out.close()

pickle_out = open('y_val.pickle', 'wb')
pickle.dump(y_val, pickle_out)  # dumps y_val into pickle_out
pickle_out.close()


In [0]:
from tensorflow.keras import layers, models
import pickle

train_images = pickle.load(open('X.pickle', 'rb'))  # the data
train_labels = pickle.load(open('y.pickle', 'rb'))  # labels for each data

test_images = pickle.load(open('X_val.pickle', 'rb'))  # the data
test_labels = pickle.load(open('y_val.pickle', 'rb'))  # labels for each data

train_images, test_images = train_images / 255.0, test_images / 255.0


In [41]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 448, 448, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 222, 222, 64)      18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 109, 109, 64)      36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 760384)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

In [42]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, 
                    validation_data=(test_images, test_labels))

Train on 600 samples, validate on 375 samples
Epoch 1/5
600/600 [==============================] - 9s 15ms/sample - loss: 3.9889 - acc: 0.3317 - val_loss: 1.0939 - val_acc: 0.3333
Epoch 2/5
600/600 [==============================] - 9s 15ms/sample - loss: 1.0965 - acc: 0.3717 - val_loss: 1.0305 - val_acc: 0.5600
Epoch 3/5
600/600 [==============================] - 9s 15ms/sample - loss: 0.8768 - acc: 0.7650 - val_loss: 0.1903 - val_acc: 1.0000
Epoch 4/5
600/600 [==============================] - 9s 15ms/sample - loss: 0.2175 - acc: 0.9433 - val_loss: 0.0171 - val_acc: 0.9947
Epoch 5/5
600/600 [==============================] - 9s 15ms/sample - loss: 0.0655 - acc: 0.9833 - val_loss: 0.0067 - val_acc: 1.0000


In [11]:
DATADIR_EVAL = "/content/drive/My Drive/PseudoDataReal"

evaluation_data = []

def create_evaluation_data():
  for category in CATEGORIES:  
    path = os.path.join(DATADIR_EVAL,category)  
    class_num = CATEGORIES.index(category)  # if 0, type 1 data; 1, type 2 data; 2, type 3 data. 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        evaluation_data.append([new_array, class_num])

create_evaluation_data()

print(len(evaluation_data))


75


In [0]:
X_eval = []
y_eval = []

In [0]:
random.shuffle(evaluation_data)

for features, label in evaluation_data:
  X_eval.append(features)
  y_eval.append(label)

X_eval = np.array(X_eval).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_eval = X_eval/255.0


In [45]:
model.evaluate(X_eval,  y_eval, verbose=1)

75/75 [==============================] - 0s 5ms/sample - loss: 0.0068 - acc: 1.0000


[0.006783944573253393, 1.0]